In [1]:
import pandas as pd
import numpy as np

## Read in the Data and Find the Most Common EPA Site Type for Each County

In [2]:
# Read in data on mandatory EPA reporting sites
# https://www.epa.gov/frs/geospatial-data-download-service
epa = pd.read_csv('NATIONAL_SINGLE.CSV')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5,7,11,13,14,15,16,21,24,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Visualize the first few rows of data
epa.head()

,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,...,SIC_CODES,SIC_CODE_DESCRIPTIONS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058360857,FORT JENSEN,NaN,NaN,1,BRISTOL BAY,02060,AK,ALASKA,...,NaN,NaN,57.885100,-157.096820,EIS,NaN,NaN,NaN,NAD83,NaN
1,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058283236,ADAK COD COOPERATIVE LLC,UNKNOWN,NaN,ADAK,ALEUTIANS WEST,NaN,AK,ALASKA,...,2091,CANNED AND CURED FISH AND SEAFOODS,51.862620,-176.637235,ICIS,GPS - UNSPECIFIED,NaN,NaN,NAD83,NaN
2,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110038070002,ADAK NAVAL AIR STATION SITE,MAIN RD,ADAK ISLAND,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,...,4952,SEWERAGE SYSTEMS,51.877500,-176.652500,SEMS,NaN,NaN,NaN,NAD83,NaN
3,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110070215939,ADAK PETROLEUM LLC BULK FUEL FACILITY,10252 TRANSIT ROAD,NaN,ADAK,ALEUTIANS WEST (CA),AK016,AK,ALASKA,...,5171,PETROLEUM BULK STATIONS AND TERMINALS,51.863056,-176.639444,ICIS,NaN,NaN,NaN,NAD83,NaN
4,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110015787683,ADAK POWER PLANT,100 HILLSIDE BLVD,NaN,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,...,"9711, OWNE",NATIONAL SECURITY,51.870300,-176.659000,EIS,UNKNOWN,NaN,NaN,NAD83,NaN


In [4]:
# Below I am having problems with county names not being recognized as the same, I think extra spaces are the reason
# so i am using the strip function to remove spaces
epa['STATE_CODE'] = epa['STATE_CODE'].str.strip()
epa['COUNTY_NAME'] = epa['COUNTY_NAME'].str.strip()
epa['SIC_CODE_DESCRIPTIONS'] = epa['SIC_CODE_DESCRIPTIONS'].str.strip()


In [5]:
# Make all columns visable
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Make a new column that will act as a counter
epa['counter']=1
epa.head(15)

,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,COUNTRY_NAME,POSTAL_CODE,FEDERAL_FACILITY_CODE,FEDERAL_AGENCY_NAME,TRIBAL_LAND_CODE,TRIBAL_LAND_NAME,CONGRESSIONAL_DIST_NUM,CENSUS_BLOCK_CODE,HUC_CODE,EPA_REGION_CODE,SITE_TYPE_NAME,LOCATION_DESCRIPTION,CREATE_DATE,UPDATE_DATE,US_MEXICO_BORDER_IND,PGM_SYS_ACRNMS,INTEREST_TYPES,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC,counter
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058360857,FORT JENSEN,NaN,NaN,1,BRISTOL BAY,02060,AK,ALASKA,UNITED STATES,00000,NaN,NaN,NaN,NaN,00,2.164000e+13,19030202.0,10.0,STATIONARY,NaN,06-APR-14,01-JUN-17,NaN,EIS:16091811,AIR EMISSIONS CLASSIFICATION UNKNOWN,NaN,NaN,NaN,NaN,57.885100,-157.096820,EIS,NaN,NaN,NaN,NAD83,NaN,1
1,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058283236,ADAK COD COOPERATIVE LLC,UNKNOWN,NaN,ADAK,ALEUTIANS WEST,NaN,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,05-APR-14,11-JAN-16,NaN,NPDES:AKU000292,ICIS-NPDES UNPERMITTED,NaN,NaN,2091,CANNED AND CURED FISH AND SEAFOODS,51.862620,-176.637235,ICIS,GPS - UNSPECIFIED,NaN,NaN,NAD83,NaN,1
2,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110038070002,ADAK NAVAL AIR STATION SITE,MAIN RD,ADAK ISLAND,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,Yes,Defense: Navy,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,18-FEB-09,01-MAR-19,NaN,"BRAC:AK4170024323, EIS:10606011, FFDOCKET:AK41...","AIR EMISSIONS CLASSIFICATION UNKNOWN, BRAC, CE...",NaN,NaN,4952,SEWERAGE SYSTEMS,51.877500,-176.652500,SEMS,NaN,NaN,NaN,NAD83,NaN,1
3,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110070215939,ADAK PETROLEUM LLC BULK FUEL FACILITY,10252 TRANSIT ROAD,NaN,ADAK,ALEUTIANS WEST (CA),AK016,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,06-APR-18,NaN,NaN,NPDES:AKR06AA03,"ICIS-NPDES NON-MAJOR, STORM WATER INDUSTRIAL",NaN,NaN,5171,PETROLEUM BULK STATIONS AND TERMINALS,51.863056,-176.639444,ICIS,NaN,NaN,NaN,NAD83,NaN,1
4,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110015787683,ADAK POWER PLANT,100 HILLSIDE BLVD,NaN,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,20-NOV-03,01-JUN-17,NaN,"AIR:AK0000000201600016, AIRS/AFS:0201600016, E...","AIR EMISSIONS CLASSIFICATION UNKNOWN, AIR MAJOR",928110,NATIONAL SECURITY.,"9711, OWNE",NATIONAL SECURITY,51.870300,-176.659000,EIS,UNKNOWN,NaN,NaN,NAD83,NaN,1
5,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110070215797,ADAK WWTP,PTARMIGAN ST,END OF ROAD,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,06-APR-18,19-SEP-18,NaN,NPDES:AKU000322,ICIS-NPDES UNPERMITTED,221320,SEWAGE TREATMENT FACILITIES.,4952,SEWERAGE SYSTEMS,51.864943,-176.629979,ICIS,NaN,NaN,NaN,NAD83,NaN,1
6,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110006876933,AEC ADAK FUEL DOCK,CROSS RD,NaN,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,NaN,Defense: Army,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,01-MAR-00,27-FEB-19,NaN,RCRAINFO:AKR000004259,SQG,NaN,NaN,NaN,NaN,51.853820,-176.654406,EPA_SLT,NaN,640.0,NaN,NAD83,NaN,1
7,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110064791060,ALASKA MARITIME NWR GREAT SITKIN ISLAND,GREAT SITKIN ISLAND,NaN,ADAK,ALEUTIANS WEST,NaN,AK,ALASKA,NaN,99546,Yes,Interior: not otherwise classified,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,16-SEP-15,13-JUL-16,NaN,FFDOCKET:AKSFN1002234,FEDERAL FACILITY HAZARDOUS WASTE DOCKET,NaN,NaN,NaN,NaN,52.053124,-176.109982,EPA_SLT,INTERPOLATION-PHOTO,33.0,FACILITY CENTROID,NAD83,NaN,1
8,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110009281124,ALASKA MARITIME NWR KISKA ISLAND USFWS,KISKA ISLAND,"51 58' 00 N, 177 33' 00 EAST"

In [6]:
# Make a copy of the epa data frame
epa_sic = epa

In [7]:
# Drop all nan values in the SIC_CODES column
epa_sic['SIC_CODE_DESCRIPTIONS'].dropna(inplace=True)

In [8]:
# A single site might have multiple LST codes and right now they are a string separated by commas, to do the 
# analysis they need to be in a list and then we can separate them out into a row for each SIC code, that way 
# we can count how many of each SIC code are in each county and get a sense of what the top EPA concerns are in each area
# i.e. mining, manufacturing etc. 
# I used this resource for this code chunk https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

lst_col = 'SIC_CODE_DESCRIPTIONS' 

epa_sic = epa_sic.assign(**{lst_col:epa_sic[lst_col].str.split(',')})

epa_sic.head()

,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,COUNTRY_NAME,POSTAL_CODE,FEDERAL_FACILITY_CODE,FEDERAL_AGENCY_NAME,TRIBAL_LAND_CODE,TRIBAL_LAND_NAME,CONGRESSIONAL_DIST_NUM,CENSUS_BLOCK_CODE,HUC_CODE,EPA_REGION_CODE,SITE_TYPE_NAME,LOCATION_DESCRIPTION,CREATE_DATE,UPDATE_DATE,US_MEXICO_BORDER_IND,PGM_SYS_ACRNMS,INTEREST_TYPES,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC,counter
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058360857,FORT JENSEN,NaN,NaN,1,BRISTOL BAY,02060,AK,ALASKA,UNITED STATES,00000,NaN,NaN,NaN,NaN,00,2.164000e+13,19030202.0,10.0,STATIONARY,NaN,06-APR-14,01-JUN-17,NaN,EIS:16091811,AIR EMISSIONS CLASSIFICATION UNKNOWN,NaN,NaN,NaN,NaN,57.885100,-157.096820,EIS,NaN,NaN,NaN,NAD83,NaN,1
1,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058283236,ADAK COD COOPERATIVE LLC,UNKNOWN,NaN,ADAK,ALEUTIANS WEST,NaN,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,05-APR-14,11-JAN-16,NaN,NPDES:AKU000292,ICIS-NPDES UNPERMITTED,NaN,NaN,2091,[CANNED AND CURED FISH AND SEAFOODS],51.862620,-176.637235,ICIS,GPS - UNSPECIFIED,NaN,NaN,NAD83,NaN,1
2,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110038070002,ADAK NAVAL AIR STATION SITE,MAIN RD,ADAK ISLAND,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,Yes,Defense: Navy,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,18-FEB-09,01-MAR-19,NaN,"BRAC:AK4170024323, EIS:10606011, FFDOCKET:AK41...","AIR EMISSIONS CLASSIFICATION UNKNOWN, BRAC, CE...",NaN,NaN,4952,[SEWERAGE SYSTEMS],51.877500,-176.652500,SEMS,NaN,NaN,NaN,NAD83,NaN,1
3,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110070215939,ADAK PETROLEUM LLC BULK FUEL FACILITY,10252 TRANSIT ROAD,NaN,ADAK,ALEUTIANS WEST (CA),AK016,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,06-APR-18,NaN,NaN,NPDES:AKR06AA03,"ICIS-NPDES NON-MAJOR, STORM WATER INDUSTRIAL",NaN,NaN,5171,[PETROLEUM BULK STATIONS AND TERMINALS],51.863056,-176.639444,ICIS,NaN,NaN,NaN,NAD83,NaN,1
4,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110015787683,ADAK POWER PLANT,100 HILLSIDE BLVD,NaN,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,20-NOV-03,01-JUN-17,NaN,"AIR:AK0000000201600016, AIRS/AFS:0201600016, E...","AIR EMISSIONS CLASSIFICATION UNKNOWN, AIR MAJOR",928110,NATIONAL SECURITY.,"9711, OWNE",[NATIONAL SECURITY],51.870300,-176.659000,EIS,UNKNOWN,NaN,NaN,NAD83,NaN,1


In [9]:
# Right now each site has a list of SIC codes which indicate what type of site it is, I want to make it one row per
# SIC code, so there might be 3 rows for a site with 3 different uses, I am using the pd.explode function for this
epa_sic = pd.DataFrame(epa_sic)
epa_sic = epa_sic.explode('SIC_CODE_DESCRIPTIONS')





In [10]:
# We have to drop nans again since they were sometimes in a list before and are now in their own cells
epa_sic['SIC_CODE_DESCRIPTIONS'].dropna(inplace=True)




In [11]:
epa_sic.head()


,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,COUNTRY_NAME,POSTAL_CODE,FEDERAL_FACILITY_CODE,FEDERAL_AGENCY_NAME,TRIBAL_LAND_CODE,TRIBAL_LAND_NAME,CONGRESSIONAL_DIST_NUM,CENSUS_BLOCK_CODE,HUC_CODE,EPA_REGION_CODE,SITE_TYPE_NAME,LOCATION_DESCRIPTION,CREATE_DATE,UPDATE_DATE,US_MEXICO_BORDER_IND,PGM_SYS_ACRNMS,INTEREST_TYPES,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC,counter
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058360857,FORT JENSEN,NaN,NaN,1,BRISTOL BAY,02060,AK,ALASKA,UNITED STATES,00000,NaN,NaN,NaN,NaN,00,2.164000e+13,19030202.0,10.0,STATIONARY,NaN,06-APR-14,01-JUN-17,NaN,EIS:16091811,AIR EMISSIONS CLASSIFICATION UNKNOWN,NaN,NaN,NaN,NaN,57.885100,-157.096820,EIS,NaN,NaN,NaN,NAD83,NaN,1
1,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058283236,ADAK COD COOPERATIVE LLC,UNKNOWN,NaN,ADAK,ALEUTIANS WEST,NaN,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,05-APR-14,11-JAN-16,NaN,NPDES:AKU000292,ICIS-NPDES UNPERMITTED,NaN,NaN,2091,CANNED AND CURED FISH AND SEAFOODS,51.862620,-176.637235,ICIS,GPS - UNSPECIFIED,NaN,NaN,NAD83,NaN,1
2,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110038070002,ADAK NAVAL AIR STATION SITE,MAIN RD,ADAK ISLAND,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,Yes,Defense: Navy,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,18-FEB-09,01-MAR-19,NaN,"BRAC:AK4170024323, EIS:10606011, FFDOCKET:AK41...","AIR EMISSIONS CLASSIFICATION UNKNOWN, BRAC, CE...",NaN,NaN,4952,SEWERAGE SYSTEMS,51.877500,-176.652500,SEMS,NaN,NaN,NaN,NAD83,NaN,1
3,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110070215939,ADAK PETROLEUM LLC BULK FUEL FACILITY,10252 TRANSIT ROAD,NaN,ADAK,ALEUTIANS WEST (CA),AK016,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,06-APR-18,NaN,NaN,NPDES:AKR06AA03,"ICIS-NPDES NON-MAJOR, STORM WATER INDUSTRIAL",NaN,NaN,5171,PETROLEUM BULK STATIONS AND TERMINALS,51.863056,-176.639444,ICIS,NaN,NaN,NaN,NAD83,NaN,1
4,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110015787683,ADAK POWER PLANT,100 HILLSIDE BLVD,NaN,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,20-NOV-03,01-JUN-17,NaN,"AIR:AK0000000201600016, AIRS/AFS:0201600016, E...","AIR EMISSIONS CLASSIFICATION UNKNOWN, AIR MAJOR",928110,NATIONAL SECURITY.,"9711, OWNE",NATIONAL SECURITY,51.870300,-176.659000,EIS,UNKNOWN,NaN,NaN,NAD83,NaN,1


In [45]:
# Each column will contain the count of that site type for each county
sic_numbers = pd.pivot_table(epa_sic, values='counter',index=['STATE_CODE','COUNTY_NAME'],columns= 'SIC_CODE_DESCRIPTIONS',aggfunc=np.sum)

In [46]:
sic_numbers.shape

(6163, 2277)

In [47]:
# Examining the columns, there
cols = sic_numbers.columns.tolist()
cols

[' ACOUSTICAL',
 ' ACTORS',
 ' ADHESIVES AND SEALANTS',
 ' ADJUSTMENT AND COLLECTION SERVICES',
 ' ADMINISTRATION OF PUBLIC HEALTH PROGRAMS',
 ' ADMINISTRATION OF SOCIAL',
 ' ADMINISTRATION OF URBAN PLANNING AND COMMUNITY AND RURAL DEVELOPMENT',
 ' ADVERTISING',
 ' ADVERTISING AGENCIES',
 ' AERONAUTICAL',
 ' AIR AND WATER RESOURCE AND SOLID WASTE MANAGEMENT',
 ' AIR COURIER SERVICES',
 ' AIR TRANSPORTATION',
 ' AIR-CONDITIONING AND WARM AIR HEATING EQUIPMENT AND COMMERCIAL AND INDUSTRIAL REFRIGERATION EQUIPMENT',
 ' AIRCRAFT',
 ' AIRCRAFT ENGINES AND ENGINE PARTS',
 ' AIRCRAFT PARTS AND AUXILIARY EQUIPMENT',
 ' AIRPORTS',
 ' ALKALIES AND CHLORINE',
 ' ALUMINUM DIE-CASTINGS',
 ' ALUMINUM EXTRUDED PRODUCTS',
 ' ALUMINUM FOUNDRIES',
 ' ALUMINUM ROLLING AND DRAWING',
 ' ALUMINUM SHEET',
 ' AMMUNITION',
 ' AMUSEMENT AND RECREATION SERVICES',
 ' AMUSEMENT PARKS',
 ' AND AGENTS FOR LAUNDRIES AND DRYCLEANERS',
 ' AND AIRPORT TERMINAL SERVICES',
 ' AND ALLIED GARMENTS',
 ' AND ALLIED PRODUCTS',

In [48]:
# I don't want 'NOT ELSEWHERE CLASSIFIED'  or 'NONCLASSIFIABLE ESTABLISHMENTS' to come up as the top 
# site type for any county so I am removing them

sic_numbers = sic_numbers.drop([' NOT ELSEWHERE CLASSIFIED',' NONCLASSIFIABLE ESTABLISHMENTS','NONCLASSIFIABLE ESTABLISHMENTS'], axis =1)


sic_numbers.head()

SIC_CODE_DESCRIPTIONS                   ACOUSTICAL   ACTORS  \
STATE_CODE COUNTY_NAME                                        
AK         ALEUTIANS EAST                      NaN      NaN   
           ALEUTIANS EAST (B)                  NaN      NaN   
           ALEUTIANS WEST                      NaN      NaN   
           ALEUTIANS WEST (CA)                 NaN      NaN   
           ALEUTIANS WEST CENSUS AREA          NaN      NaN   

SIC_CODE_DESCRIPTIONS                   ADHESIVES AND SEALANTS  \
STATE_CODE COUNTY_NAME                                           
AK         ALEUTIANS EAST                                  NaN   
           ALEUTIANS EAST (B)                              NaN   
           ALEUTIANS WEST                                  NaN   
           ALEUTIANS WEST (CA)                             NaN   
           ALEUTIANS WEST CENSUS AREA                      NaN   

SIC_CODE_DESCRIPTIONS                   ADJUSTMENT AND COLLECTION SERVICES  \
STATE_CODE COUNTY_NAME                                                       
AK         ALEUTIANS EAST                                              NaN   
           ALEUTIANS EAST (B)                                          NaN   
           ALEUTIANS WEST                                              NaN   
           ALEUTIANS WEST (CA)                                         NaN   
           ALEUTIANS WEST CENSUS AREA                                  NaN   

SIC_CODE_DESCRIPTIONS                   ADMINISTRATION OF PUBLIC HEALTH PROGRAMS  \
STATE_CODE COUNTY_NAME                                                             
AK         ALEUTIANS EAST                                                    NaN   
           ALEUTIANS EAST (B)                                                NaN   
           ALEUTIANS WEST                                                    NaN   
           ALEUTIANS WEST (CA)                                               NaN   
           ALEUTIANS WEST CENSUS AREA                                        NaN   

SIC_CODE_DESCRIPTIONS                   ADMINISTRATION OF SOCIAL  \
STATE_CODE COUNTY_NAME                                             
AK         ALEUTIANS EAST                                    NaN   
           ALEUTIANS EAST (B)                                NaN   
           ALEUTIANS WEST                                    NaN   
           ALEUTIANS WEST (CA)                               NaN   
           ALEUTIANS WEST CENSUS AREA                        NaN   

SIC_CODE_DESCRIPTIONS                   ADMINISTRATION OF URBAN PLANNING AND COMMUNITY AND RURAL DEVELOPMENT  \
STATE_CODE COUNTY_NAME                                                                                         
AK         ALEUTIANS EAST                                                            NaN                       
           ALEUTIANS EAST (B)                                                        NaN                       
           ALEUTIANS WEST                                                            NaN                       
           ALEUTIANS WEST (CA)                                                       NaN                       
           ALEUTIANS WEST CENSUS AREA                                                NaN                       

SIC_CODE_DESCRIPTIONS                   ADVERTISING   ADVERTISING AGENCIES  \
STATE_CODE COUNTY_NAME                                                       
AK         ALEUTIANS EAST                       NaN                    NaN   
           ALEUTIANS EAST (B)                   NaN                    NaN   
           ALEUTIANS WEST                       NaN                    NaN   
           ALEUTIANS WEST (CA)                  NaN                    NaN   
           ALEUTIANS WEST CENSUS AREA           NaN                    NaN   

SIC_CODE_DESCRIPTIONS                   AERONAUTICAL  \
STATE_CODE COUNTY_NAME                                 
AK         ALEUTIANS EAST             

In [49]:
sic_numbers.shape

(6163, 2274)

In [50]:
new_df = sic_numbers

In [53]:
# Now we want to find which column has the highest value for each county that is what is the most common type of site
# in each county

top_site_type = pd.DataFrame(new_df.idxmax(axis=1))

In [54]:
top_site_type.head(2500)

0
STATE_CODE COUNTY_NAME                                                                           
AK         ALEUTIANS EAST                              PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           ALEUTIANS EAST (B)                          PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           ALEUTIANS WEST                              PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           ALEUTIANS WEST (CA)                                 CANNED AND CURED FISH AND SEAFOODS
           ALEUTIANS WEST CENSUS AREA                  PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           ANCHORAGE                                                     EXCEPT ELEVATED HIGHWAYS
           ANCHORAGE MUNICIPALITY                                                       GOLD ORES
           BALDWIN COUNTY                                              FABRICATED RUBBER PRODUCTS
           BETHEL                                                                SEWERAGE SYSTEMS
           BETHEL (CA)                                                                  GOLD ORES
           BETHEL CENSUS AREA                                                           GOLD ORES
           BRISTOL BAY                                 PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           COBB COUNTY                                              COMPUTER PROGRAMMING SERVICES
           COLUMBIA COUNTY                                        GENERAL WAREHOUSING AND STORAGE
           DENALI                                                                       GOLD ORES
           DENALI BOROUGH                                                               GOLD ORES
           DILLINGHAM                                                           ELECTRIC SERVICES
           DILLINGHAM (CA)                                     CANNED AND CURED FISH AND SEAFOODS
           FAIRBANKS                                                                    GOLD ORES
           FAIRBANKS NORTH STAR                                                         GOLD ORES
           FAIRBANKS NORTH STAR BOROUGH                                                 GOLD ORES
           HAINES                                                    CONSTRUCTION SAND AND GRAVEL
           HAINES BOROUGH                                                               GOLD ORES
           HOONAH ANGOON                                                         SEWERAGE SYSTEMS
           HOONAH-ANGOON                                                         SEWERAGE SYSTEMS
           HOONAH-ANGOON CENSUS AREA                            AND ELEVATED HIGHWAY CONSTRUCTION
           JUNEAU                                                                SEWERAGE SYSTEMS
           JUNEAU CITY AND BOROUGH                                       EXCEPT ELEVATED HIGHWAYS
           KENAI PENINSULA                                        CRUDE PETROLEUM AND NATURAL GAS
           KENAI PENINSULA BOROUGH                                                      GOLD ORES
           KETCHIKAN GATEWAY                                                     SEWERAGE SYSTEMS
           KETCHIKAN GATEWAY BOROUGH                                                      LOGGING
           KING COUNTY                                                                  GOLD ORES
           KODIAK ISLAND                               PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           KODIAK ISLAND BOROUGH                                                   PHOSPHATE ROCK
           LAKE AND PENINSULA                          PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           MATANUSKA SUSITNA                                                   HEAVY CONSTRUCTION
           MATANUSKA-SUSITNA                                                            GOLD ORES
           MATANUSKA-SUSITNA BOROUGH                                                    GOLD ORES
           MATANUSKA-SUSITNA D                                                

In [55]:
top_site_type.reset_index(inplace=True)

## Find Out How Many EPA Reporting Sites There are In Each County

In [56]:
# I want to get a dataframe that is just the name of the county and the number of sites in that county
# I will use pivot tables
county_numbers = pd.pivot_table(epa, values='counter',index=['STATE_CODE','COUNTY_NAME'],aggfunc=np.sum)

In [57]:
county_numbers.head(100)

counter
STATE_CODE COUNTY_NAME                                 
AK         ALASKA                                     1
           ALEUTIAN ISLANDS                           5
           ALEUTIANS EAST                            62
           ALEUTIANS EAST (B)                         1
           ALEUTIANS EAST BOROUGH                     3
           ALEUTIANS WEST                            88
           ALEUTIANS WEST (CA)                       20
           ALEUTIANS WEST CENSUS AREA                29
           ANCHORAGE                               1285
           ANCHORAGE MUNICIPALITY                   442
           BALDWIN COUNTY                             1
           BARROW COUNTY                              4
           BETHEL                                   228
           BETHEL (CA)                               29
           BETHEL CENSUS AREA                        11
           BRISTOL BAY                               73
           BRISTOL BAY BOROUGH                        3
           CLAYTON COUNTY                             2
           COBB COUNTY                                1
           COLUMBIA COUNTY                            8
           COWETA                                     1
           DEKALB COUNTY                              4
           DENALI                                    97
           DENALI BOROUGH                            17
           DILLINGHAM                                73
           DILLINGHAM (CA)                           12
           DILLINGHAM CENSUS AREA                     9
           DODGE                                      1
           FAIRBANKS                                  3
           FAIRBANKS NORTH STAR                     902
           FAIRBANKS NORTH STAR BOROUGH              24
           FAYETTE                                    1
           FORSYTH                                    1
           FULTON COUNTY                             94
           GWINNETT COUNTY                            3
           HAINES                                    53
           HAINES BOROUGH                             7
           HALL COUNTY                                4
           HENRY COUNTY                              12
           HOONAH ANGOON                              8
           HOONAH-ANGOON                             43
           HOONAH-ANGOON CENSUS AREA                 18
           JACKSON COUNTY                             1
           JENKINS COUNTY                             1
           JUNEAU                                   220
           JUNEAU CITY AND                           13
           JUNEAU CITY AND BOROUGH                   79
           KENAI PENINSULA                          959
           KENAI PENINSULA BOROUGH                   27
           KENIA                                      1
           KETCHIKAN GATEWAY                        254
           KETCHIKAN GATEWAY BOROUGH                  7
           KING COUNTY                                2
           KODIAK ISLAND                            200
           KODIAK ISLAND BOROUGH                      3
           KUSILVAK                                   2
           LAKE AND PENINSULA                       104
           LAKE AND PENINSULA BOROUGH                 1
           LAURENS                                    1
           LOWNDES COUNTY                             1
           MATANUSKA SUSITNA                        104
           MATANUSKA-SUSITNA                       1446
           MATANUSKA-SUSITNA BOROUGH                 87
           MATANUSKA-SUSITNA D                        1
           MULTIPLE (PORTABLE FACILITIES)            10
           NOME                                     160
           NOME (CA)                                219
           NOME CENSUS AREA                          32
           NORTH SLOPE                              391
           NORTH SLOPE (B)                            1
           NORTH

In [58]:
# Lets get a look at what the distribution is like

county_numbers.describe()

,counter
count,7994.000000
mean,474.232299
std,2058.777070
min,1.000000
25%,4.000000
50%,39.000000
75%,333.000000
max,98458.000000


## Join the Tables So Each County Has a Count and Top Site Type

In [59]:
# Check the shape
county_numbers.shape

(7994, 1)

In [60]:
# Check the shape
top_site_type.shape

(6163, 3)

In [61]:
# Looks like there is not a top site type for every county, so we wan to do a left join so that missing ones will
# be filled in with NaN and we won't lose an county data
result = county_numbers.merge(top_site_type, how='left', on=['STATE_CODE','COUNTY_NAME'])


In [62]:
# Check that there are 7994 rows and not 6163 to make sure that the merge worked as expected
result.shape

(7994, 4)

In [63]:
result.head(100)

,STATE_CODE,COUNTY_NAME,counter,0
0,AK,ALASKA,1,NaN
1,AK,ALEUTIAN ISLANDS,5,NaN
2,AK,ALEUTIANS EAST,62,PREPARED FRESH OR FROZEN FISH AND SEAFOODS
3,AK,ALEUTIANS EAST (B),1,PREPARED FRESH OR FROZEN FISH AND SEAFOODS
4,AK,ALEUTIANS EAST BOROUGH,3,NaN
5,AK,ALEUTIANS WEST,88,PREPARED FRESH OR FROZEN FISH AND SEAFOODS
6,AK,ALEUTIANS WEST (CA),20,CANNED AND CURED FISH AND SEAFOODS
7,AK,ALEUTIANS WEST CENSUS AREA,29,PREPARED FRESH OR FROZEN FISH AND SEAFOODS
8,AK,ANCHORAGE,1285,EXCEPT ELEVATED HIGHWAYS
9,AK,ANCHORAGE MUNICIPALITY,442,GOLD ORES


In [65]:
# Export the results to a csv that can be imported into Tableau
result.to_csv('df.csv')

## Lets Add Some Points of Interest to the Map

In [ ]:
# Data on EPA powerplant sites
# https://www.epa.gov/enviro/facility-registry-service-frs
